In [23]:
from bs4 import BeautifulSoup, SoupStrainer
import requests
import urllib.request
import copy
import pandas as pd
from newspaper import Article
import re
import nltk
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
import string
import pandas as pd
import re 
# словарь для отнесения к теме
data = pd.read_csv('vocab.csv')
stopwords_ru=list(data['0'])
# символы для удаления
patterns = "[!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
morph = MorphAnalyzer()

In [34]:
# новостей про ИБ с кждого сайта 
it_news_count=200
# новостей не про ИБ с каждого сайта
simpl_news_count=200
# фэйков генерить
fake_news=40

In [35]:
# лемматизация
def lemmatize(doc):
    doc = re.sub(patterns, ' ', doc)
    tokens = []
#     print("len=",len(doc.split()))
    for token in doc.split():
        if token and token not in stopwords_ru:
            token = token.strip()
            token = morph.normal_forms(token)[0]
            tokens.append(token)
    raznost=len(doc.split())-len(tokens)
#     print("/////////////////////////////////////")
#     print(raznost)
#     print(round(0.2*len(doc.split())))
#     print()
    if  raznost >= round(0.05*len(doc.split())):
        return 0
    else:
        return 112

In [36]:
# получить текст новостной статьи и проверяем его на принадлежность теме
def get_news(url, data_,flag,datc):
    news = Article(url, language='ru')
    news.download()
    news.parse()
    data_frame=pd.DataFrame()
    el=str(news.text)
    if flag==1:
        text = copy.deepcopy(el[el.find("\n",el.find("\n")+2)+2:].lower())
    else:
        text=copy.deepcopy(el)
    data_frame=pd.DataFrame()
    data_frame=pd.concat([data_frame,pd.DataFrame([text])])
    flag=data_frame[0].apply(lemmatize)
    if (datc==0 or datc==1) and flag[0]==0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[1],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,0
    elif (datc==0 or datc==2) and flag[0]!=0:
        data_=pd.concat([data_,pd.DataFrame({'News':[copy.deepcopy(el)], 'InfSec':[0],'FAKE?':[0],'Link':[copy.deepcopy(url)]})])
        return data_,112
    return None,None
# 0 - текст новости
# 1 - относится ли к ИБ (1-да, 0- нет)
# 2 - является фейком (1-да,0 - нет)
# 3 - ссылка на новость

In [37]:
def flag_func(flag,it_news,simpl_news,datchik):
    #     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    global it_news_count
    global simpl_news_count
    if flag==0:
        it_news=it_news+1
    else:
        simpl_news=simpl_news+1
    if datchik==0:
        if it_news>=it_news_count:
            datchik=2
        if simpl_news>=simpl_news_count:
            datchik=1
    return it_news,simpl_news,datchik

In [38]:
# выясняем сколько страниц на сайие
def secure_lab_page():
    url="https://www.securitylab.ru/news/page1_1.php"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('div', class_='page-picker')
    st = str(box.find('input'))
    num_page=''
    for i in range(12,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [39]:
# получаем новости
def secure_lab_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
#     for k in range(6,8):
        url="https://www.securitylab.ru/news/page1_"+str(k)+".php"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
        box = soup.find_all('a', class_='article-card inline-card')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
#                 if k==7:
#                     print(box[i])
                urls=''
                for j in range (42,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         print("https://www.securitylab.ru"+urls)
#                         print(datchik)
                        temp_dataframe,flag=get_news("https://www.securitylab.ru"+urls,dataframe,1,datchik)
#                         print(flag)
                        if flag!=None:
#                             print(urls)
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [40]:
# выясняем сколько страниц на сайие
def xakep_page():
    url="https://xakep.ru/category/news/"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div'))
    box = soup.find('span', class_='pages')
    st=str(box)
    num_page=''
    for i in range(34,len(st)):
        if st[i]=='<':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [41]:
def xakep_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(1,num_page+1):
        url="https://xakep.ru/category/news/page/"+str(k)+"/"
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='bdaia-blocks-container'))
        box = soup.find_all('h3',class_='entry-title')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (33,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news(urls,dataframe,2)
                        temp_dataframe,flag=get_news(urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [42]:
# выясняем сколько страниц на сайие
def anti_malware_page():
    url="https://www.anti-malware.ru/news?page=0"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('li',class_='pager-last'))
    box = soup.find('a')
    st=str(box)
    num_page=''
    for i in range(20,len(st)):
        if st[i]=='"':
            break
        elif re.search(r'[0-9]', st[i]):
            num_page=num_page+st[i]
    num_page=int(num_page)
    return num_page

In [43]:
def anti_malware_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://www.anti-malware.ru/news?page="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='region region-content'))
        box = soup.find_all('h2')
        link=[]
        for i in range (0, len(box)):
            try:
                stroka_url=str(box[i])
                urls=''
                for j in range (13,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news('https://www.anti-malware.ru'+urls,dataframe,2)
                        temp_dataframe,flag=get_news('https://www.anti-malware.ru'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [44]:
# выясняем сколько страниц на сайие
def securitymedia_page():
    url="https://securitymedia.org/news/?PAGEN_2=1"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
    result = requests.get(url, headers=headers)
    # поиск по тегам
    soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='modern-page-navigation'))
    box = soup.find_all('a')
    st=str(box[len(box)-2])
    num_page=''
    for i in range(24,len(st)):
        if st[i]=='"':
            break
        else:
            num_page=num_page+st[i]
#     print(num_page)
    num_page=int(num_page)
    return num_page

In [45]:
def securitymedia_news(num_page):
    global it_news_count
    global simpl_news_count
    it_news=0
    simpl_news=0
#     отображает какие новости добавлять: 0-все, 1-ИБ, 2-простые
    datchik=0
    dataframe=pd.DataFrame()
    for k in range(0,num_page):
        url="https://securitymedia.org/news/?PAGEN_2="+str(k)
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:100.0) Gecko/20100101 Firefox/100.0'}
        result = requests.get(url, headers=headers)
        soup = BeautifulSoup(result.text, 'html.parser', parse_only=SoupStrainer('div',class_='col-md-9'))
        box = soup.find_all('a',class_="text-dark")
        link=[]
        for i in range (0, len(box),2):
            try:
                stroka_url=str(box[i])
                urls=''
                
                for j in range (27,len(stroka_url)):
                    if stroka_url[j]!='"':
                        urls=urls+stroka_url[j]
                    else:
#                         dataframe=get_news('https://securitymedia.org'+urls,dataframe,2)
                        temp_dataframe,flag=get_news('https://securitymedia.org'+urls,dataframe,2,datchik)
                        if flag!=None:
                            dataframe=copy.deepcopy(temp_dataframe)
                        break
                if flag!=None:
                    it_news,simpl_news,datchik=flag_func(flag,it_news,simpl_news,datchik)
                    if it_news>=it_news_count and simpl_news>=simpl_news_count:
                        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
                        return dataframe
            except Exception:
                e=1
        print('len dataframe=',len(dataframe),' it_news=',it_news,' simpl_news=',simpl_news,"  странициа сайта=",k)
    return dataframe

In [46]:
result_data1frame=pd.DataFrame()
result_data2frame=pd.DataFrame()
result_data3frame=pd.DataFrame()
result_data4frame=pd.DataFrame()

In [48]:
page=securitymedia_page()
# org
site_pd=securitymedia_news(copy.deepcopy(page))
result_data1frame=pd.concat([site_pd,result_data1frame])

len dataframe= 20  it_news= 13  simpl_news= 7   странициа сайта= 0
len dataframe= 40  it_news= 26  simpl_news= 14   странициа сайта= 1
len dataframe= 60  it_news= 42  simpl_news= 18   странициа сайта= 2
len dataframe= 80  it_news= 59  simpl_news= 21   странициа сайта= 3
len dataframe= 100  it_news= 76  simpl_news= 24   странициа сайта= 4
len dataframe= 120  it_news= 94  simpl_news= 26   странициа сайта= 5
len dataframe= 140  it_news= 112  simpl_news= 28   странициа сайта= 6
len dataframe= 160  it_news= 127  simpl_news= 33   странициа сайта= 7
len dataframe= 180  it_news= 143  simpl_news= 37   странициа сайта= 8
len dataframe= 200  it_news= 158  simpl_news= 42   странициа сайта= 9
len dataframe= 220  it_news= 172  simpl_news= 48   странициа сайта= 10
len dataframe= 240  it_news= 187  simpl_news= 53   странициа сайта= 11
len dataframe= 260  it_news= 200  simpl_news= 60   странициа сайта= 12
len dataframe= 263  it_news= 200  simpl_news= 63   странициа сайта= 13
len dataframe= 268  it_news

In [49]:
page=anti_malware_page()
# тут страницы с нулевой
site_pd=anti_malware_news(copy.deepcopy(page+1))
result_data2frame=pd.concat([site_pd,result_data2frame])

len dataframe= 10  it_news= 9  simpl_news= 1   странициа сайта= 0
len dataframe= 20  it_news= 16  simpl_news= 4   странициа сайта= 1
len dataframe= 30  it_news= 21  simpl_news= 9   странициа сайта= 2
len dataframe= 40  it_news= 28  simpl_news= 12   странициа сайта= 3
len dataframe= 50  it_news= 36  simpl_news= 14   странициа сайта= 4
len dataframe= 60  it_news= 45  simpl_news= 15   странициа сайта= 5
len dataframe= 70  it_news= 53  simpl_news= 17   странициа сайта= 6
len dataframe= 80  it_news= 62  simpl_news= 18   странициа сайта= 7
len dataframe= 90  it_news= 71  simpl_news= 19   странициа сайта= 8
len dataframe= 100  it_news= 80  simpl_news= 20   странициа сайта= 9
len dataframe= 110  it_news= 86  simpl_news= 24   странициа сайта= 10
len dataframe= 120  it_news= 95  simpl_news= 25   странициа сайта= 11
len dataframe= 130  it_news= 104  simpl_news= 26   странициа сайта= 12
len dataframe= 140  it_news= 113  simpl_news= 27   странициа сайта= 13
len dataframe= 150  it_news= 121  simpl_n

len dataframe= 384  it_news= 200  simpl_news= 184   странициа сайта= 115
len dataframe= 385  it_news= 200  simpl_news= 185   странициа сайта= 116
len dataframe= 386  it_news= 200  simpl_news= 186   странициа сайта= 117
len dataframe= 387  it_news= 200  simpl_news= 187   странициа сайта= 118
len dataframe= 388  it_news= 200  simpl_news= 188   странициа сайта= 119
len dataframe= 388  it_news= 200  simpl_news= 188   странициа сайта= 120
len dataframe= 389  it_news= 200  simpl_news= 189   странициа сайта= 121
len dataframe= 391  it_news= 200  simpl_news= 191   странициа сайта= 122
len dataframe= 392  it_news= 200  simpl_news= 192   странициа сайта= 123
len dataframe= 394  it_news= 200  simpl_news= 194   странициа сайта= 124
len dataframe= 395  it_news= 200  simpl_news= 195   странициа сайта= 125
len dataframe= 396  it_news= 200  simpl_news= 196   странициа сайта= 126
len dataframe= 396  it_news= 200  simpl_news= 196   странициа сайта= 127
len dataframe= 397  it_news= 200  simpl_news= 197  

In [47]:
page=xakep_page()
site_pd=xakep_news(copy.deepcopy(page))
result_data3frame=pd.concat([site_pd,result_data3frame])

len dataframe= 40  it_news= 32  simpl_news= 8   странициа сайта= 1
len dataframe= 80  it_news= 62  simpl_news= 18   странициа сайта= 2
len dataframe= 120  it_news= 95  simpl_news= 25   странициа сайта= 3
len dataframe= 160  it_news= 124  simpl_news= 36   странициа сайта= 4
len dataframe= 200  it_news= 150  simpl_news= 50   странициа сайта= 5
len dataframe= 240  it_news= 185  simpl_news= 55   странициа сайта= 6
len dataframe= 262  it_news= 200  simpl_news= 62   странициа сайта= 7
len dataframe= 269  it_news= 200  simpl_news= 69   странициа сайта= 8
len dataframe= 279  it_news= 200  simpl_news= 79   странициа сайта= 9
len dataframe= 295  it_news= 200  simpl_news= 95   странициа сайта= 10
len dataframe= 304  it_news= 200  simpl_news= 104   странициа сайта= 11
len dataframe= 316  it_news= 200  simpl_news= 116   странициа сайта= 12
len dataframe= 323  it_news= 200  simpl_news= 123   странициа сайта= 13
len dataframe= 336  it_news= 200  simpl_news= 136   странициа сайта= 14
len dataframe= 34

In [50]:
page=secure_lab_page()
site_pd=secure_lab_news(copy.deepcopy(page))
result_data4frame=pd.concat([site_pd,result_data4frame])

len dataframe= 15  it_news= 15  simpl_news= 0   странициа сайта= 1
len dataframe= 30  it_news= 26  simpl_news= 4   странициа сайта= 2
len dataframe= 45  it_news= 39  simpl_news= 6   странициа сайта= 3
len dataframe= 60  it_news= 51  simpl_news= 9   странициа сайта= 4
len dataframe= 75  it_news= 63  simpl_news= 12   странициа сайта= 5
len dataframe= 90  it_news= 76  simpl_news= 14   странициа сайта= 6
len dataframe= 105  it_news= 88  simpl_news= 17   странициа сайта= 7
len dataframe= 120  it_news= 101  simpl_news= 19   странициа сайта= 8
len dataframe= 135  it_news= 113  simpl_news= 22   странициа сайта= 9
len dataframe= 150  it_news= 127  simpl_news= 23   странициа сайта= 10
len dataframe= 165  it_news= 142  simpl_news= 23   странициа сайта= 11
len dataframe= 180  it_news= 156  simpl_news= 24   странициа сайта= 12
len dataframe= 195  it_news= 168  simpl_news= 27   странициа сайта= 13
len dataframe= 210  it_news= 181  simpl_news= 29   странициа сайта= 14
len dataframe= 225  it_news= 194

In [75]:
result_data1frame.to_csv('dataset1_news.csv', encoding='utf-8', index=False)
result_data2frame.to_csv('dataset2_news.csv', encoding='utf-8', index=False)
result_data3frame.to_csv('dataset3_news.csv', encoding='utf-8', index=False)
result_data4frame.to_csv('dataset4_news.csv', encoding='utf-8', index=False)

In [53]:
result_dataframe=pd.DataFrame()
result_dataframe=pd.concat([result_dataframe,result_data1frame])
result_dataframe=pd.concat([result_dataframe,result_data2frame])
result_dataframe=pd.concat([result_dataframe,result_data3frame])
result_dataframe=pd.concat([result_dataframe,result_data4frame])

In [76]:
result_dataframe.to_csv('dataset_news.csv', encoding='utf-8', index=False)

In [77]:
len(result_dataframe)

1600

In [14]:
d1 = pd.read_csv('dataset1.csv',on_bad_lines='skip',sep='\t')
d2= pd.read_csv('dataset2.csv',on_bad_lines='skip',sep='\t')
d3 = pd.read_csv('dataset3.csv',on_bad_lines='skip',sep='\t')
d4 = pd.read_csv('dataset6.csv')

In [15]:
d=pd.DataFrame()
d=pd.concat([d,d1])
d=pd.concat([d,d2])
d=pd.concat([d,d3])
d=pd.concat([d,d4])
d=d.head(400)

In [16]:
d.to_csv('dataset_fake.csv', encoding='utf-8', index=False)

In [17]:
d

,News,InfSec,FAKE?,Link
0,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
1,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
2,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
3,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
4,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
...,...,...,...,...
125,['Новость: ФСТЭК опубликовал требования по обе...,1,1,https://fake_news.ru
126,['Новость: ФСТЭК опубликовал новые правила защ...,1,1,https://fake_news.ru
127,['Новость: ФСТЭК опубликовал методику проведен...,1,1,https://fake_news.ru
128,['Новость: ФСТЭК опубликовал требования к защи...,1,1,https://fake_news.ru


In [41]:
y= pd.read_csv('dataset_fake.csv')
y

,News,InfSec,FAKE?,Link
0,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
1,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
2,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
3,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
4,['Новость: специалисты по информационной безоп...,1,1,https://fake_news.ru
...,...,...,...,...
395,['Новость: ФСТЭК опубликовал требования по обе...,1,1,https://fake_news.ru
396,['Новость: ФСТЭК опубликовал новые правила защ...,1,1,https://fake_news.ru
397,['Новость: ФСТЭК опубликовал методику проведен...,1,1,https://fake_news.ru
398,['Новость: ФСТЭК опубликовал требования к защи...,1,1,https://fake_news.ru


In [48]:
y= pd.read_csv('dataset_fake_corrrect.csv')
pat="[0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
for i in range (0, len(y['News'])):
    u=y['News'].iloc[i]
    u=u.replace('Новость: специалисты по информационной безопасности заявили', '')
    u=u.replace('[', ' ')
    u=u.replace(']', ' ')
    u=u.replace("'", ' ')
    u=u.replace('Новость: специалисты по информационной безопасности заявили', '')
    u=u.replace('Новость: в ФСБ опровергли сообщения', '')
    u=u.replace('Новость: взломаны сервера одной из крупнейших компаний', '')
    u=u.replace('Новость: ФСТЭК опубликовал', '')
    u=u.replace('Новость: Российсике хакеры снова', '')
    u=u.replace('Новость: ФСТЭК вводит новые', '')
    u=u.replace('Alexander Antipov', '')
    u=u.replace("_", '')
    u=u.replace('+', '')
    u=u.replace('=', '')
    u=u.replace('✂', '')
    u=u.replace('#', '')
    u=u.replace(',', ' ')
    u=u.replace('(', ' ')
    u=u.replace('..', ' ')
    u=re.sub(pat, ' ', u)
    u=re.sub('\n', ' ', u)
    u=re.sub('\t', ' ', u)
    u=re.sub('[A-Za-z]', ' ', u)
    y.loc[i,'News']=copy.deepcopy(u)
y.to_csv('dataset_fake_corrrect.csv', encoding='utf-8', index=False)

In [33]:
y.to_csv('dataset_fake_corrrect.csv', encoding='utf-8', index=False)

In [37]:
y= pd.read_csv('dataset_fake_corrrect.csv')
pat="[a-zA-Z0-9!#$%&'()*+,./:;<=>?@[\]^_`{|}~—\"\-]+"
for i in range (0, len(y['News'])):
    u=y['News'].iloc[i]
    u=u.replace('Новость: специалисты по информационной безопасности заявили', '')
    u=u.replace('[', '')
    u=u.replace(']', '')
    u=u.replace("'", '')
    u=u.replace('Новость: специалисты по информационной безопасности заявили', '')
    u=u.replace('Новость: в ФСБ опровергли сообщения', '')
    u=u.replace('Новость: взломаны сервера одной из крупнейших компаний', '')
    u=u.replace('Новость: ФСТЭК опубликовал', '')
    u=u.replace('Новость: Российсике хакеры снова', '')
    u=u.replace('Новость: ФСТЭК вводит новые', '')
    u=u.replace('Alexander Antipov', '')
    u=u.replace('', '')
    u=u.replace('', '')
    u=u.replace("_", '')
    u=u.replace('+', '')
    u=u.replace('=', '')
    u=u.replace('✂', '')
    u=u.replace('#', '')
    u=u.replace(',', '')
    u=u.replace('(', '')
    u=u.replace('..', '')
    u=re.sub(pat, '', u)
    u=re.sub('\n', '', u)
    u=re.sub('\t', '', u)
    y.loc[i,'News']=copy.deepcopy(u)
y.to_csv('dataset_fake_corrrect.csv', encoding='utf-8', index=False)

In [52]:
y= pd.read_csv('dataset_fake_corrrect.csv')
len(y)

400

In [60]:
test_dataset=pd.DataFrame()
result_data1frame=pd.read_csv('dataset1_news.csv')
result_data2frame=pd.read_csv('dataset2_news.csv')
result_data3frame=pd.read_csv('dataset3_news.csv')
result_data4frame=pd.read_csv('dataset4_news.csv')
y= pd.read_csv('dataset_fake_corrrect.csv')
print(len(y))
test_dataset=pd.concat([test_dataset,result_data1frame.head(50)])
test_dataset=pd.concat([test_dataset,result_data1frame.tail(50)])

test_dataset=pd.concat([test_dataset,result_data2frame.head(50)])
test_dataset=pd.concat([test_dataset,result_data2frame.tail(50)])

test_dataset=pd.concat([test_dataset,result_data3frame.head(50)])
test_dataset=pd.concat([test_dataset,result_data3frame.tail(50)])

test_dataset=pd.concat([test_dataset,result_data4frame.head(50)])
test_dataset=pd.concat([test_dataset,result_data4frame.tail(50)])
print(len(test_dataset))
# for i in range (0, len(test_dataset['News'])):
#     u=test_dataset['News'].iloc[i]
#     u=u.replace('Alexander Antipov', '')
#     test_dataset.loc[i,'News']=copy.deepcopy(u)
print(len(test_dataset))
test_dataset=pd.concat([test_dataset,y]).reset_index(drop=True)

test_dataset.to_csv('corrrect_dataset.csv', encoding='utf-8', index=False)

400
400
400


In [61]:
test_dataset

,News,InfSec,FAKE?,Link
0,Чаще всего целями кибератак становятся госучре...,0,0,https://securitymedia.org/news/lk-nazvala-sfer...
1,26 мая состоялось очередное заседание Обществе...,0,0,https://securitymedia.org/news/rkn-predlozhil-...
2,Вышедшее на этой неделе превью опциональных на...,0,0,https://securitymedia.org/news/obnovlenie-dlya...
3,Human Rights Watch (HRW) изучила более 150 про...,1,0,https://securitymedia.org/news/sotni-prilozhen...
4,Торговая площадка баз данных Industrial Spy за...,1,0,https://securitymedia.org/news/torgovaya-plosh...
...,...,...,...,...
795,требования по обеспечению безопасности инфо...,1,1,https://fake_news.ru
796,новые правила защиты персональных данных в ...,1,1,https://fake_news.ru
797,методику проведения оценки соответствия сре...,1,1,https://fake_news.ru
798,требования к защите персональных данных при...,1,1,https://fake_news.ru
